# Etude pour la représentation d'isochrones en 3D

> Pour utiliser ce notebook, vous aurez besoin de quelques librairies Python. Il est préférable de les installer via Anaconda:
* bokeh>=1.0.2
* colour>=0.1.5
* geojson>=2.3.0
* geopandas>=0.3.0
* ipyvolume>=0.5.1
* numpy>=1.14.2
* pyproj>=1.9.5.1
* shapely>=1.6.4

## Problématique

> Pour produire les isochrones nous nous basons sur les modules Python de ce dépôt qui utilisent eux-mêmes l'API de Navitia. 

Il est question d'utiliser l'altitude comme transposition d'un paramètre. Il s'agit par exemple de représenter le temps via une altitude. Par exemple, on peut imaginer des isochrones générés à partir d'une origine avec une durée fixe mais des requêtes effectuées à différents moments de la journée. 

Prenons un exemple concret pour être encore plus précis avec ces paramètres:
* **une adresse précise comme origine**: *"90, Boulevard Saint-Germain, 75005, France"*,
* **une durée fixe**: *20 minutes*
* **des requêtes multiples**: de 6:00 à 11:00 toutes les 15 minutes par exemple

Néanmoins, dans un premier temps, nous allons tester de cartographier des polygone simples - en l'occurrence des cercles - tout d'abord en 2D puis en 3D. 

## Première étape: créer et cartographier des cercles
### Quelques paramètres

* **start**: limite inférieure pour la création *random* de cercles
* **end**: limite supérieure pour la création *random* de cercles
* **elevation**: *hauteur* unique pour chaque cercle
* **geojson_file**: chemin et nom du fichier GeoJSON à créer et comportant les cercles
* **max_distance**: distance maximale en mètres, soit le rayon maximum pour un cercle


In [1]:
#Params
start = 0
end = 10
elevation = 50 #in meters
geojson_file = "polys.geojson"
max_distance = 20000

### Quelques fonctions 

Il s'agit de fonctions permettant de transformer des GeoDataframes vers les formats Bokeh, et des formats Bokeh vers les formats nécessaires pour l'affichage avec la librairie Ipyvolume. 

In [2]:
from shapely.ops import triangulate
from shapely.geometry import MultiPoint

def get_poly_coords(poly):
    
    if poly.geom_type == "Polygon":
        if poly.interiors:
            xs_coords = [
                [poly.exterior.xy[0].tolist()],
                [[xy[0] for xy in p.coords] for p in poly.interiors]
            ]

            ys_coords =  [
                [poly.exterior.xy[1].tolist()],
                [[xy[1] for xy in p.coords] for p in poly.interiors]
            ]
        else:
            xs_coords = [
                [poly.exterior.xy[0].tolist()]
            ]

            ys_coords =  [
                [poly.exterior.xy[1].tolist()]
            ]
        
    elif poly.geom_type == "MultiPolygon":
        xs_coords = []
        ys_coords = []
        for po in poly:
            if po.interiors:
                xs_coords.extend([
                    [po.exterior.xy[0].tolist()],
                    [[xy[0] for xy in p.coords] for p in po.interiors]]
                )

                ys_coords.extend([
                    [po.exterior.xy[1].tolist()],
                    [[xy[1] for xy in p.coords] for p in po.interiors]]
                )
            else:
                xs_coords.extend([
                    [po.exterior.xy[0].tolist()]
                ])

                ys_coords.extend([
                    [po.exterior.xy[1].tolist()]
                ])
    
    return [xs_coords], [ys_coords]

def multi_gdf_to_multi_bokeh(gdf):
    dico = {}
    xs, ys = [], []
    gdf["coords"] = gdf.apply(lambda x: get_poly_coords(x["geometry"]), axis=1)
    l = gdf["coords"].values.tolist()
    
    columns = gdf.columns.tolist()
    columns.remove("geometry")
    columns.remove("coords")
    
    for column in columns:
        dico[column] = gdf[column].values.tolist()
    
    for coord in l:
        xs.append(coord[0][0])
        ys.append(coord[1][0])
        
    dico.update(
        {
            "xs": xs,
            "ys": ys
        }
    )
        
    return dico

def get_vertices(xs, ys, min_alt, max_alt):
    XS, YS, alt = [], [], []
    
    for i,x in enumerate(xs):
        XS.append([x, x])
    for i,y in enumerate(ys):
        YS.append([y, y])
        
    for i in range(0,len(XS)):
        alt.append([min_alt, max_alt])
        
    return np.array(XS), np.array(YS), np.array(alt)

# def get_surface(xs, ys, min_alt, max_alt):
#     #Triangulation
#     points = MultiPoint([(x,y) for x,y in zip(xs,ys)])
#     triangles = triangulate(points)
    
#     for triangle in triangles:
#         xs = triangle.exterior.coords.xy[0]
#         ys = triangle.exterior.coords.xy[1]
        

def get_limits(array):
    return array.min(), array.max()

In [ ]:
for i,x in enumerate(xs[:-1]):
    XS.append([x, xs[i+1]])
    
for i,x in enumerate(ys[:-1]):
    YS.append([x, ys[i+1]])

for i in range(0,len(XS)):
        alt.append([1, 1])

## Une première étape: cartographier des polygones simples
### Afficher en 2D avec Bokeh

On construit des cercles qu'on va tout d'abord cartographier en utilisant la librairie Bokeh.

> Il est possible d'activer/désactiver les couches en cliquant dans la légende mais également d'utiliser les widgets sur le côté de la carte:
* pan
* zoom
* export (*PNG*)

In [3]:
from shapely.geometry import Point
import random
import geopandas as gpd
import os
from pyproj import Proj, transform


##Set origin
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
lat,lng = 2.34472, 48.85103 #EPSG 4326
lat2,lng2 = transform(inProj,outProj,lat,lng)

circles = []

#Create dictionary ...
for i in range(start,end):
    r = lambda: random.randint(0,255)
    circles.append(
        {
            "geometry":Point(
                lat2,
                lng2
            ).buffer(
                random.randint(1,max_distance) #1 minimum to avoid empty polygone with 0 as distance
            ),
            "elevation": elevation,
            "color": '#%02X%02X%02X' % (r(),r(),r())
        },
    )

#... then make a GeoDataframe
gdf = gpd.GeoDataFrame(circles)

#Export to GeoJSON
try:
    os.remove(geojson_file)
except OSError:
    pass

gdf.to_file(geojson_file, driver="GeoJSON")

source_polys = multi_gdf_to_multi_bokeh(gdf)

#Make dict for each circle (to build layers later)
dico_test = {}
columns = source_polys.keys()
for i in range(start,end):
    if i not in dico_test.keys():
        dico_test[i] = {}
    for col in columns:
        dico_test[i][col] = source_polys[col][i]


In [4]:
import geopandas as gpd
import geojson
from bokeh.plotting import figure, show
from bokeh.palettes import Viridis256, Magma11, RdYlBu11, Greens9, Inferno256, Viridis6
from bokeh.io import output_notebook
from bokeh.tile_providers import STAMEN_TONER_BACKGROUND
from bokeh.models import GeoJSONDataSource, ColumnDataSource, CategoricalColorMapper, LinearColorMapper
from bokeh.palettes import Colorblind8 as cb
from bokeh.transform import factor_cmap
import json
from pyproj import Proj, transform

output_notebook()


#Prepare the Bokeh figure
# color_mapper = LinearColorMapper(palette=Viridis4)
p = figure(
    title="Circles"
)

p.width = 800
p.add_tile(STAMEN_TONER_BACKGROUND, alpha=0.2)


#Set origin
data_ori = {
    "x":[lat2],
    "y":[lng2],
    "name":["Origin"]
}

ori = ColumnDataSource(data_ori)

#Prepare layer for each circle
for name,value in dico_test.items():
    p.patches(
    xs=value['xs'][0],
    ys=value['ys'][0],
    fill_color=value['color'],
    line_color=value['color'],
    fill_alpha=0.0,
    line_alpha=1,
    line_width = 4,
    legend=str(name)
    )
    
#Add ori
p.triangle(
    x="x",
    y="y",
    size=20,
    color="red",
    alpha=0.5,
    source=ori,
    legend="Origin"
)

p.legend.click_policy="hide"

show(p)

Loading BokehJS ...

### Essai en 3D avec Ipyvolume

> Il est possible de manipuler le cube contenant les cercles en 3D:
* zoom avec la molette de la souris
* rotations avec le clic gauche et le mouvement de la souris

In [5]:
import numpy as np
import ipyvolume as ipv

elevation = 10
alt_min = 0

min_xs, min_ys, max_xs, max_ys = [], [], [], []

figure1 = ipv.figure(width=800, height=600)

for k,v in dico_test.items():
    alt_max = alt_min + elevation
    
    XS = dico_test[k]["xs"][0][0]
    YS = dico_test[k]["ys"][0][0]

    xs, ys, alt = get_vertices(XS, YS, alt_min, alt_max)
        
    minx, maxx = get_limits(xs)
    miny, maxy = get_limits(ys)
    minalt, maxalt = get_limits(alt)
    
    min_xs.append(minx)
    min_ys.append(miny)
    max_xs.append(maxx)
    max_ys.append(maxy)

    ipv.plot_surface(xs, ys, alt, color=dico_test[k]["color"])
    
    ipv.xlim(min(min_xs), max(max_xs))
    ipv.ylim(min(min_ys), max(max_ys))
    ipv.zlim(0, alt_max)
        
    alt_min = alt_max
    
ipv.show()

## Une deuxième étape: cartographier des isochrones en 3D
### Isochrones avec Ipyvolume

In [6]:
test = "polys_5h_11h.geojson"
test_gdf = gpd.GeoDataFrame.from_file(test, driver="GeoJSON")

polys = multi_gdf_to_multi_bokeh(test_gdf)

start,end = 0, len(polys["id"])

#Make dict for each poly (to build layers later)
dico_polys = {}
columns = polys.keys()
for i in range(start,end):
    if i not in dico_polys.keys():
        dico_polys[i] = {}
    for col in columns:
        dico_polys[i][col] = polys[col][i]

In [7]:
import numpy as np
import ipyvolume as ipv
from IPython.display import display, HTML
from colour import Color
import ipywidgets as widgets

out = widgets.Output()

first = Color("#440154")
last = Color("#FDE724")
colors = list(first.range_to(last, len(dico_polys.keys())))
colors = [color.hex for color in colors]

elevation = 5
alt_min = 0

min_xs, min_ys, max_xs, max_ys = [], [], [], []
i = 0

figure2 = ipv.figure(width=800, height=600)

for k,v in dico_polys.items():
    alt_max = alt_min + elevation
    
    XS = v["xs"][0][0]
    YS = v["ys"][0][0]

    xs, ys, alt = get_vertices(XS, YS, alt_min, alt_max)
    
    
    minx, maxx = get_limits(xs)
    miny, maxy = get_limits(ys)
    minalt, maxalt = get_limits(alt)
    
    min_xs.append(minx)
    min_ys.append(miny)
    max_xs.append(maxx)
    max_ys.append(maxy)

    ipv.plot_surface(xs, ys, alt, color=colors[i])
    
    ipv.xlim(min(min_xs), max(max_xs))
    ipv.ylim(min(min_ys), max(max_ys))
    ipv.zlim(0, alt_max)
        
    alt_min = alt_max
    
    i += 1
    
#Make a legend
html = '<table><thread><thread><tr><th colspan="1">LEGEND</th></tr></thead><tbody>'
for key,color in zip(polys["time"], colors):
    tmp = '<tr><td bgcolor={}><font color="#FFFFFF"><b>{}</b></font></td></tr>'.format(color,key)
    html = html + tmp
html = html + "</tbody></table>"

html_widget = widgets.HTML(
    value=html,
    placeholder="right"
)

box = widgets.HBox([html_widget, figure2])
out.append_display_data(box)
display(out)

Output(outputs=({'output_type': 'display_data', 'data': {}, 'metadata': {}},))

### Isolignes bufferisées avec Ipyvolume

In [11]:
import numpy as np
import ipyvolume as ipv
from IPython.display import display, HTML
from colour import Color
import ipywidgets as widgets

out = widgets.Output()

test = "isolines.geojson"
test_gdf = gpd.GeoDataFrame.from_file(test, driver="GeoJSON")

polys = multi_gdf_to_multi_bokeh(test_gdf)

start,end = 0, len(polys["id"])

#Make dict for each poly (to build layers later)
dico_polys = {}
columns = polys.keys()
for i in range(start,end):
    if i not in dico_polys.keys():
        dico_polys[i] = {}
    for col in columns:
        dico_polys[i][col] = polys[col][i]

first = Color("#440154")
last = Color("#FDE724")
colors = list(first.range_to(last, len(dico_polys.keys())))
colors = [color.hex for color in colors]

elevation = 5
alt_min = 0

min_xs, min_ys, max_xs, max_ys = [], [], [], []
i = 0

figure3 = ipv.figure(width=800, height=600)

for k,v in dico_polys.items():
    alt_max = alt_min + elevation
    
    XS = v["xs"][0][0]
    YS = v["ys"][0][0]

    xs, ys, alt = get_vertices(XS, YS, alt_min, alt_max)
    
    minx, maxx = get_limits(xs)
    miny, maxy = get_limits(ys)
    minalt, maxalt = get_limits(alt)
    
    min_xs.append(minx)
    min_ys.append(miny)
    max_xs.append(maxx)
    max_ys.append(maxy)

    ipv.plot_surface(xs, ys, alt, color=colors[i])
    
    ipv.xlim(min(min_xs), max(max_xs))
    ipv.ylim(min(min_ys), max(max_ys))
    ipv.zlim(0, alt_max)
        
    alt_min = alt_max
    
    i += 1
    
#Make a legend
html = '<table><thread><thread><tr><th colspan="1">LEGEND</th></tr></thead><tbody>'
for key,color in zip(polys["time"], colors):
    tmp = '<tr><td bgcolor={}><font color="#FFFFFF"><b>{}</b></font></td></tr>'.format(color,key)
    html = html + tmp
html = html + "</tbody></table>"

html_widget = widgets.HTML(
    value=html,
    placeholder="right"
)

widgets.HBox([html_widget, figure3])
# out.append_display_data(box)
# display(out)

In [12]:
#Création d'une grille en prenant la bbox du polygone
#Découpe de la grille par le polygone
#Récupération des intersections (éventuellement création de points dans un certain rayon)
#Scatter plot ipyvolume

import geopandas as gpd
import numpy as np
import pandas as pd
import osmnx as ox

test = "isolines.geojson"
test_gdf = gpd.GeoDataFrame.from_file(test, driver="GeoJSON")

bounds = test_gdf.bounds


In [13]:
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon

cols = 100
rows = 100

def make_dense_pts(minx, maxx, miny, maxy, num=100, alt=0.0, color="#0000ff", epsg="epsg:4326"):
    pts = []
    array_xs = np.linspace(minx, maxx, num=num)
    array_ys = np.linspace(miny, maxy, num=num)
#     array_alt = np.array([alt for i in range(0, array_xs.size)])
    
    for y in array_ys:
        for x in array_xs:
            pts.append(
                {
                    "geometry":Point(x,y),
                    "alt": alt,
                    "color": color
                },
            )
    
    points = gpd.GeoDataFrame(pts)
    points.crs = {"init":epsg}
    
    return array_xs, array_ys, points


def get_pts_within(geometry, pts):
    """
    
    Based on: https://github.com/gboeing/urban-data-science/blob/master/19-Spatial-Analysis-and-Cartography/rtree-spatial-indexing.ipynb
    """
    pts_within = pd.DataFrame()
    spatial_index = pts.sindex

    if isinstance(geometry, Polygon):
        geometry = MultiPolygon([geometry])

    geometry_cut = ox.quadrat_cut_geometry(geometry, quadrat_width=0.1)

    # find the points that intersect with each subpolygon and add them to pts_within
    for poly in geometry_cut:
        possible_matches_index = list(spatial_index.intersection(poly.bounds))
        possible_matches = pts.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(poly)]

        pts_within = pts_within.append(precise_matches)
        
    return pts_within

def get_arrays(points, alt):
    
    return points.geometry.x.values, points.geometry.y.values, np.array(
        [
            alt for i in range(
                0, points.geometry.x.values.size
            )
        ]
    )

def get_layers(xs, ys, alt, min_, max_, step):
    x, y, z = [], [], []
    random_size = xs.size//2
    
    for i in np.arange(min_, max_, step):
        xs_add = np.random.choice(xs, random_size)
        ys_add = np.random.choice(ys, random_size)
        x= np.concatenate((x, xs_add), axis=0)
        y= np.concatenate((y, ys_add), axis=0)
        tmp_alt = float(min_ + i)
        z = np.concatenate(
            (
                z, np.array([tmp_alt for j in range(0, xs.size)])
            ),
            axis=0
        )
    
    return x, y, z


In [14]:
import geopandas as gpd
import numpy as np
import pandas as pd
import osmnx as ox
from colour import Color

test = "isolines.geojson"
test_gdf = gpd.GeoDataFrame.from_file(test, driver="GeoJSON")

bounds = test_gdf.bounds
minx, maxx, miny, maxy = bounds.iloc[0]["minx"], bounds.iloc[0]["maxx"], bounds.iloc[0]["miny"], bounds.iloc[0]["maxy"]
array_xs, array_ys, points = make_dense_pts(minx, maxx, miny, maxy, num=20)

size = test_gdf.index.max()
first = Color("#440154")
last = Color("#FDE724")
colors = list(first.range_to(last, size))

elevation = 0 

XS, YS, ALT, COLORS = [], [], [], []

for i in range(0, size):
    geometry = test_gdf['geometry'].iloc[i]
    pts_within = get_pts_within(geometry, points)
    xs, ys, alt = get_arrays(pts_within, float(elevation))
    XS = np.concatenate((XS, xs), axis=0)
    YS = np.concatenate((YS, ys), axis=0)
    ALT = np.concatenate((ALT, alt), axis=0)
    color = list(colors[i].get_rgb())
    COLORS.extend(color for j in range(0, xs.size))
    elevation += 1


In [16]:
import numpy as np
import ipyvolume as ipv
from IPython.display import display, HTML
import ipywidgets as widgets

# out = widgets.Output()

ipv.figure()

# scatter = ipv.figure(width=800, height=600)
scatter = ipv.quickscatter(
    XS, 
    YS, 
    ALT, 
    size=4, 
    marker="sphere", 
    color=COLORS
)
    
ipv.xlim(minx, maxx)
ipv.ylim(miny, maxy)
ipv.zlim(0.0, float(size))

#Make a legend

html = '<table><thread><thread><tr><th colspan="1">LEGEND</th></tr></thead><tbody>'
for key,color in zip(reversed(test_gdf["time"]), reversed([color.hex for color in colors])):
    tmp = '<tr><td bgcolor={}><font color="#FFFFFF"><b>{}</b></font></td></tr>'.format(color,key)
    html = html + tmp
html = html + "</tbody></table>"

html_widget = widgets.HTML(
    value=html,
    placeholder="right"
)

widgets.HBox([html_widget, scatter])

# ipv.show()

In [ ]:
from scipy.spatial import Delaunay

roof_delaunay = scipy.spatial.Delaunay(XS)
roof_model = p3.plot_trisurf(df_extra['Y_r'],df_extra['Z'],df_extra['X_r'], triangles=roof_delaunay.simplices, color='red')
fig = ipv.figure(width=800, height=600)

fig.meshes.append(roof_model)

In [ ]:
from scipy.spatial import Delaunay, ConvexHull
import scipy 

fig = ipv.figure(width=800, height=600)
pts = np.dstack((XS,YS))
roof_delaunay = scipy.spatial.Delaunay(pts[0])

roof_model = ipv.plot_trisurf(XS,ALT,YS, triangles=roof_delaunay.simplices, color='red')
fig.meshes.append(roof_model)

ipv.show()

In [ ]:
from scipy.spatial import Delaunay, ConvexHull
import scipy 

pts = np.dstack((XS,YS, ALT))
hull = ConvexHull(pts[0])

In [ ]:
ipv.plot_surface(XS, ALT, YS, color="orange")

In [ ]:
test_xs, test_ys, test_alt = [],[],[]

for i,x in enumerate(XS[:-1]):
    test_xs.append([x, XS[i+1]])
for i,x in enumerate(YS[:-1]):
    test_ys.append([x, YS[i+1]])
for i,x in enumerate(ALT[:-1]):
    test_alt.append([x, ALT[i+1]])

n_xs = np.array(test_xs)
n_ys = np.array(test_ys)
n_alt = np.array(test_alt)

In [ ]:
from scipy.spatial import Delaunay

points = np.array(
    [
        [1,1],[1,2],[2,2],[2,1]
    ]
)

tri = Delaunay(points)

In [ ]:
triangles = points[tri.simplices]


In [ ]:
a = np.arange(-5, 5)
U, V = np.meshgrid(a, a)
X = U
Y = V
Z = X*Y**2

ipv.figure()
ipv.plot_surface(X, Z, Y, color="orange")
ipv.plot_wireframe(X, Z, Y, color="red")
ipv.show()

In [17]:
from scipy.spatial import Delaunay

tri = Delaunay(np.array([XS,YS]).T)

# ipv.figure()
# delaunay = Delaunay(pts)
# ground = ipv.plot_trisurf(XS, YS, ALT, triangles = delaunay.simplices, color='lightgrey')
ipv.plot_trisurf(XS, YS, ALT, triangles=tri.simplices, color="red")

ipv.xlim(minx, maxx)
ipv.ylim(miny, maxy)
ipv.zlim(0.0, float(size))

ipv.show()

In [24]:
contours_x, contours_y, contours_alt = [],[],[]
elevation = 5
alt_min = 0

ipv.figure()

for k,v in dico_polys.items():
    alt_max = alt_min + elevation
    contours_x.extend(v["xs"][0][0])
    contours_y.extend(v["ys"][0][0])
    
    for i in range(0,len(v["xs"][0][0])):
        contours_alt.extend([alt_max])
        
    alt_min = alt_max

contours_x, contours_y, contours_alt = np.array(contours_x), np.array(contours_y), np.array(contours_alt), 

tri = Delaunay(np.array([contours_x,contours_y]).T)

ipv.plot_trisurf(contours_x, contours_y, contours_alt, triangles=tri.simplices, color="red")

ipv.xlim(minx, maxx)
ipv.ylim(miny, maxy)
ipv.zlim(0.0, float(size))

ipv.show() 

In [25]:
print (contours_x.size)
print (contours_y.size)
print (contours_alt.size)

364884
364884
364884
